In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sb
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

In [ ]:
data= pd.read_csv("tweets.csv")

In [ ]:
data.head()


In [ ]:
data.shape

In [ ]:
data.drop(columns=['id'],axis=1,inplace=True)
data.head()


In [ ]:
#check the class  distribution
data["label"].value_counts()

In [ ]:
plt.bar([0,1],data["label"].value_counts())
plt.title("class proportions in the dataset")
plt.show()

In [ ]:
dff = data.drop(['label'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(dff,list(data.label), test_size=0.1)


In [ ]:
X_test.shape

In [ ]:
X_temp['label'] = y_temp

In [ ]:

X_temp.head()

In [ ]:
type(X_temp)

In [ ]:
nonhate = X_temp[X_temp['label'] == 0]

In [ ]:
nonhate.head()

In [ ]:
hate = X_temp[X_temp.label == 1]

In [ ]:
hate.shape

In [ ]:
nonhatesample = nonhate.sample(n = hate.shape[0])

In [ ]:
nonhatesample.head()

In [ ]:
nonhatesample.shape

In [ ]:
ds = pd.concat([hate, nonhatesample], axis = 0)

In [ ]:
ds.shape

In [ ]:
testdf = X_test
testdf['label'] = y_test

In [ ]:
testdf.shape

In [ ]:

ds = pd.concat([ds, testdf], axis = 0)

In [ ]:
ds.head()

In [ ]:
ds.shape

In [ ]:
#check the class  distribution
ds["label"].value_counts()

In [ ]:
plt.bar([0,1],ds["label"].value_counts())
plt.title("class proportions in the dataset")
plt.show()

In [ ]:
ds.to_csv("tweets.csv")

In [ ]:
ds = pd.read_csv("tweets.csv")

In [ ]:
ds.head()

In [ ]:
#Code to remove @
#ds['tweet'] = ds['tweet'].apply(lambda x : ' '.join([tweet for tweet in x.split()if not tweet.startswith("@")]))

In [ ]:
#ds.head()

In [ ]:
#Removing numbers
ds['tweet'] = ds['tweet'].apply(lambda x : ' '.join([tweet for tweet in x.split() if not tweet == '\d*']))

In [ ]:
ds.head()

In [ ]:
#removing # sign and &
def clean(text):
  text=re.sub(r"#"," ",text)
  text=re.sub(r"&[a-z]+"," ",text)
  text=re.sub(r"@[a-zA-Z]+"," ",text )
  return text

In [ ]:
ds["tweet"]=ds["tweet"].apply(clean)
ds.head()

In [ ]:
#Code for removing slang words
d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
               'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
               'cud':'could','fud':'food'} ## Need a huge dictionary
words = "I luv kashmir"
words = words.split()
reformed = [d[word] if word in d else word for word in words]
reformed = " ".join(reformed)

In [ ]:
reformed

In [ ]:
ds['tweet'] = ds['tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))

In [ ]:
#Removing stopwords
ds['tweet'] = ds['tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word in set(stopwords.words('english'))]))

In [ ]:
ds.head(10)

In [ ]:
#Stemming
ps = PorterStemmer()
ds['tweet'] = ds['tweet'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))

In [ ]:
ds.head()

In [ ]:
ds.to_csv("tweets.csv")

In [ ]:
data=pd.read_csv("tweets.csv")

In [ ]:
data.shape

In [ ]:
data=data.loc[:,["tweet","label"]]

In [ ]:
data.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X = cv.fit_transform(data["tweet"].values.astype('U')).toarray()

In [ ]:
X

In [ ]:
from sklearn.metrics import f1_score , accuracy_score , confusion_matrix
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,data["label"],random_state=5, test_size=0.15)

In [ ]:

from sklearn.metrics import f1_score , accuracy_score , confusion_matrix

In [ ]:
#Training model using Logistic Regression

from sklearn.linear_model import LogisticRegression
model= LogisticRegression()
model.fit(X_train,y_train)

pred= model.predict(X_test)

log_f1=f1_score(y_test,pred)
log_acc=accuracy_score(y_test,pred)
log_con=confusion_matrix(y_test,pred)

In [ ]:
 #Training model using Random Forest classifier

from sklearn.ensemble import RandomForestClassifier
randmodel=RandomForestClassifier()
randmodel.fit(X_train,y_train)

randpred=randmodel.predict(X_test)

rand_f1=f1_score(randpred,y_test)
rand_acc=accuracy_score(randpred,y_test)
rand_con=confusion_matrix(y_test,randpred)

In [ ]:
# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train, y_train)

mulpred=spam_detect_model.predict(X_test)

mul_f1=f1_score(mulpred,y_test)
mul_acc=accuracy_score(mulpred,y_test)
mul_con=confusion_matrix(y_test,mulpred)

In [ ]:
# Training model using support vector machine

from sklearn import svm
clf=svm.SVC()
clf.fit(X_train,y_train)

svmpred=clf.predict(X_test)

svm_f1=f1_score(svmpred,y_test)
svm_acc=accuracy_score(svmpred,y_test)
svm_con=confusion_matrix(y_test,svmpred)

In [ ]:
import seaborn as sn

In [ ]:
sn.heatmap(log_con)
plt.title("Logistic Regression")

In [ ]:
sn.heatmap(rand_con)
plt.title("Random Forest")

In [ ]:
sn.heatmap(mul_con)
plt.title("Naive Bayes")

In [ ]:
sn.heatmap(svm_con)
plt.title("support vector machine")

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

print('Logistic Regression Score: ', roc_auc_score(y_test, pred))
print('Naive Bayes Score: ', roc_auc_score(y_test, mulpred))
print('Random Forest Score: ', roc_auc_score(y_test, randpred))
print('support vector machine score: ', roc_auc_score(y_test,svmpred))

In [ ]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy

import cartopy
import libarchive
import pydot
from sklearn.metrics import roc_curve

log_fpr, log_tpr, threshold = roc_curve(y_test,pred)
nav_fpr, nav_tpr, threshold = roc_curve(y_test, mulpred )
rand_fpr, rand_tpr, thresold = roc_curve(y_test, randpred)
svm_fpr, svm_tpr, threshold = roc_curve(y_test,svmpred )

In [ ]:
def graph_roc_curve_multiple(rand_fpr, rand_tpr, nav_fpr, nav_tpr, log_fpr , log_tpr,svm_fpr, svm_tpr):
    plt.figure(figsize=(8,6))
    plt.title('ROC Curve \n Classifiers', fontsize=18)
    plt.plot(rand_fpr, rand_tpr, label='random forest')
    plt.plot(nav_fpr, nav_tpr, label='Naive Bayes')
    plt.plot(log_fpr, log_tpr, label='Logistic Regression')
    plt.plot(svm_fpr, svm_tpr, label='svm')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.annotate('Minimum ROC Score of 50% \n (This is the minimum score to get)', xy=(0.5, 0.5), xytext=(0.6, 0.3),
                arrowprops=dict(facecolor='#6E726D', shrink=0.05),
                )
    plt.legend()
    
graph_roc_curve_multiple(rand_fpr, rand_tpr, nav_fpr, nav_tpr, log_fpr, log_tpr,svm_fpr, svm_tpr )
plt.show()

In [ ]:
# Using term frequency _inverse document frequency
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data["tweet"].values.astype('U')).toarray()

In [ ]:
X

In [ ]:
from sklearn.metrics import f1_score , accuracy_score , confusion_matrix
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,data["label"],random_state=5, test_size=0.15)

In [ ]:
#Training model using Logistic Regression

from sklearn.linear_model import LogisticRegression
model= LogisticRegression()
model.fit(X_train,y_train)

pred= model.predict(X_test)

log_f1=f1_score(y_test,pred)
log_acc=accuracy_score(y_test,pred)
log_con=confusion_matrix(y_test,pred)

In [ ]:
 #Training model using Random Forest classifier

from sklearn.ensemble import RandomForestClassifier
randmodel=RandomForestClassifier()
randmodel.fit(X_train,y_train)

randpred=randmodel.predict(X_test)

rand_f1=f1_score(randpred,y_test)
rand_acc=accuracy_score(randpred,y_test)
rand_con=confusion_matrix(y_test,randpred)

In [ ]:
# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB()
spam_detect_model.fit(X_train, y_train)

mulpred=spam_detect_model.predict(X_test)

mul_f1=f1_score(mulpred,y_test)
mul_acc=accuracy_score(mulpred,y_test)
mul_con=confusion_matrix(y_test,mulpred)

In [ ]:
# Training model using support vector machine

from sklearn import svm
clf=svm.SVC()
clf.fit(X_train,y_train)

svmpred=clf.predict(X_test)

svm_f1=f1_score(svmpred,y_test)
svm_acc=accuracy_score(svmpred,y_test)
svm_con=confusion_matrix(y_test,svmpred)

In [ ]:
import seaborn as sn

In [ ]:
sn.heatmap(log_con)
plt.title("Logistic Regression")

In [ ]:
sn.heatmap(rand_con)
plt.title("Random Forest")

In [ ]:
sn.heatmap(mul_con)
plt.title("Naive Bayes")

In [ ]:
sn.heatmap(svm_con)
plt.title("support vector machine")

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

print('Logistic Regression Score: ', roc_auc_score(y_test, pred))
print('Naive Bayes Score: ', roc_auc_score(y_test, mulpred))
print('Random Forest Score: ', roc_auc_score(y_test, randpred))
print('support vector machine score: ', roc_auc_score(y_test,svmpred))

In [ ]:
import cartopy
import libarchive
import pydot
from sklearn.metrics import roc_curve

log_fpr, log_tpr, threshold = roc_curve(y_test,pred)
nav_fpr, nav_tpr, threshold = roc_curve(y_test, mulpred )
rand_fpr, rand_tpr, thresold = roc_curve(y_test, randpred)
svm_fpr, svm_tpr, threshold = roc_curve(y_test,svmpred )

In [ ]:
def graph_roc_curve_multiple(rand_fpr, rand_tpr, nav_fpr, nav_tpr, log_fpr , log_tpr,svm_fpr, svm_tpr):
    plt.figure(figsize=(8,6))
    plt.title('ROC Curve \n Classifiers', fontsize=18)
    plt.plot(rand_fpr, rand_tpr, label='random forest')
    plt.plot(nav_fpr, nav_tpr, label='Naive Bayes')
    plt.plot(log_fpr, log_tpr, label='Logistic Regression')
    plt.plot(svm_fpr, svm_tpr, label='svm')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.annotate('Minimum ROC Score of 50% \n (This is the minimum score to get)', xy=(0.5, 0.5), xytext=(0.6, 0.3),
                arrowprops=dict(facecolor='#6E726D', shrink=0.05),
                )
    plt.legend()
    
graph_roc_curve_multiple(rand_fpr, rand_tpr, nav_fpr, nav_tpr, log_fpr, log_tpr,svm_fpr, svm_tpr )
plt.show()